In [1]:
import os

In [2]:
%pwd

'f:\\Unversity\\Project\\TimeSeries\\Kalimati-tarkari\\notebook'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Unversity\\Project\\TimeSeries\\Kalimati-tarkari'

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    final_data: Path

In [7]:
from kalimati_tarkari.constants import *
from kalimati_tarkari.utils.main_utils import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            final_data=config.final_data
        )

        return data_transformation_config

In [9]:
import os
from kalimati_tarkari.logger import logging
import pandas as pd

In [11]:
from datetime import datetime, timedelta

In [10]:
df = pd.read_csv('artifacts\data_ingestion\data.csv')

In [12]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.devanagari_to_english = {
        '०': '0', '१': '1', '२': '2', '३': '3', '४': '4',
        '५': '5', '६': '6', '७': '7', '८': '8', '९': '9'
        }

    def replace_devanagari_with_eng(self,text):
        for dev, eng in self.devanagari_to_english.items():
            text = text.replace(dev, eng)
        return text

    def cleaning_data(self,data_path):
        data = pd.read_csv(self.config.data_path)
         # Generate the range of dates for the year 2024
        date_range = []
        current = datetime(2013, 4, 15)
        end_date = datetime.now()
        while current <= end_date:
            date_range.append(current.strftime('%m/%d/%Y'))
            current += timedelta(days=1)

        date_range = pd.Index(date_range)
        complete_df = pd.DataFrame(date_range, columns=['Date'])
        final_df = pd.merge(complete_df,data, on=['Date'], how='left')
        final_df['Average'] = final_df['Average'].str.replace("रू","")
        final_df['Average'] = final_df['Average'].astype('str').apply(self.replace_devanagari_with_eng).astype('float')
        final_df['Date'] = pd.to_datetime(final_df['Date'])
        final_df['Date'] = final_df['Date'].dt.date
        final_df.set_index('Date', inplace=True)
        return final_df
        
        


    
    def get_data_transform(self):
        final_df = self.cleaning_data(self.config.data_path)
        missing = final_df.isnull().sum()
        logging.info(missing)
        final_df['Average'].fillna(final_df["Average"].ewm(span=60, adjust=False).mean(),inplace=True)
        logging.info("filling Missing value done sucessfully !")
        return final_df.to_csv("artifacts/data_transformation/salesDaily.csv",index=True)
        
        
    
    


In [14]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.get_data_transform()
except Exception as e:
    raise e

C:\Users\bishw\AppData\Local\Temp\ipykernel_10476\1226866887.py:42: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df['Average'].fillna(final_df["Average"].ewm(span=60, adjust=False).mean(),inplace=True)
